<a href="https://colab.research.google.com/github/decouflet/7506R-2C2022-GRUPO15/blob/main/TP1_orga_del_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
bvcn vgbc ngcgfc fgc bgv dsdsdsd